# CNN

이미지 인식, 음성 인식에 주로 사용

기존 완전 연결 계층(Affine) : 데이터를 모두 1차원 데이터로 처리해서 입력한다 --> 형상에 담긴 정보를 살릴 수 없다.

합성곱 계층(CNN) : 입력 데이터의 형상을 유지    

CNN의 필터 : 가중치

CNN의 편향 : 편향 (항상 1X1, broadcasting)

패딩의 사용이유 : 합성곱 연산을 되풀이 할 때, 출력의 형상이 계속 작아지는 것을 방지하기 위해서 사용한다. 입력데이터의 형상을 유지할 수 있다.

출력의 크기 = (입력의 크기 + 2*패딩 - 필터의 크키)/스트라이드 + 1

최대 풀링(max_pooling) : 윈도우 안에서 최대값을 추출함. 보통 스트라이드와 같은 값으로 설정

### 4차원 배열 구현

In [2]:
#형상이 (10,1,28,28)

import numpy as np

x = np.random.rand(10,1,28,28)
x.shape

(10, 1, 28, 28)

In [3]:
#총 10개의 데이터 중 첫번째 데이터에 접근

x[0].shape

(1, 28, 28)

In [4]:
#첫 번째 데이터의 첫 채널의 공간 데이터에 접근

x[0][0]

array([[2.67323325e-01, 5.28299854e-02, 2.35462172e-01, 4.22341302e-01,
        8.68327559e-01, 6.42163209e-01, 4.40922379e-01, 7.48205905e-01,
        8.98978166e-01, 6.59315857e-02, 5.62897295e-01, 1.07889308e-01,
        8.46317161e-01, 9.64903645e-02, 5.38295796e-01, 5.07893734e-01,
        1.58717468e-01, 6.48355912e-01, 3.34330797e-01, 1.00657919e-01,
        4.81839145e-01, 8.56303383e-01, 9.08611577e-01, 9.14805889e-01,
        7.05628049e-01, 3.41255974e-01, 7.28419891e-01, 7.91179140e-01],
       [9.69968652e-01, 4.85132411e-01, 7.58914123e-01, 8.29388212e-01,
        7.89012617e-01, 4.83485012e-01, 8.58916826e-01, 6.67838145e-01,
        1.95792260e-01, 9.86105718e-01, 8.72675503e-01, 5.68982819e-02,
        8.82903803e-01, 6.01643104e-01, 5.91651128e-01, 9.63822131e-02,
        2.48236315e-01, 2.18626453e-01, 8.31611465e-02, 3.00297979e-01,
        7.19986063e-01, 4.42859964e-01, 2.51558849e-01, 3.59876667e-01,
        9.83850829e-01, 7.48719225e-01, 2.55257971e-01, 4.98794

In [5]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    Parameters
    ----------
    input_data : (データ数, チャンネル, 高さ, 幅)の4次元配列からなる入力データ
    filter_h : フィルターの高さ
    filter_w : フィルターの幅
    stride : ストライド
    pad : パディング
    Returns
    -------
    col : 2次元配列
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


im2col : 입력 데이터(3차원, 배치까지 4차원)를 2차원 행렬로 변화시켜줌.

im2col(input_data,filter_h,filter_w,stride =1, pad=0)으로 사용

In [6]:
import sys,os
sys.path.append(os.pardir)

x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1,5,5,stride = 1, pad = 0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2,5,5,stride=1,pad=0)
print(col2.shape)

(9, 75)
(90, 75)


### 합성곱 계층 구현

In [7]:
class Convolution:
    def __init__(self,W,b,stride = 1,pad = 0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self,x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1+(H+2*self.pad - FH)/self.stride)
        out_w = int(1+(W + 2*self.pad - FW)/self.stride)
        
        col = im2col(x,FH,FW,self.stride,self.pad)
        col_W =self.W.reshape(FN,-1).T
        out = np.dot(col,col_W) + self.b
        
        out = out.reshape(N,out_h,out_w,-1).transpose(0,3,1,2)  #transpose(0,3,1,2) : 다차언의 축 순서를 바꿔준다.(인덱스에 따라서)
        
        return out

역전파 구현시에는 col2im 함수 사용

### 폴링 계층 구현

In [9]:
# 과정 : 입력 데이터 전개 -> 행별 최댓값 구함 -> 적절한 모양으로 성형

class Pooling:
    def __init__(self,pool_h,pool_w,stride =1,pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h)/self.stride)
        out_w = int(1+ (W - self.pool_w)/self.stride)
        
        col = im2col(x,self.pool_h,self.pool_w,self.stride, self.pad)
        col = col.reshape(-1,self.pool_h*self.pool_w)
        
        out = np.max(col,axis=1)
        
        out = out.reshape(N,out_h,out_w,C).transpose(0,3,1,2)
        
        return out

### CNN 구현

In [11]:
class SimpleConNet:
    def __init__(self,input_dim = (1,28,28),conv_param={'filter_num':30,'filter_size':5,'pad':0,'stride':1},hidden_size = 100, output_size = 10,weight_init_std = 0.01):
        filter_num = conv_param['filter_num']
        fliter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size / 2)*(conv_output_size/2))
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random_randn(filter_num,input_dim[0],filter_size,filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random_randn(pool_output_size,hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random_randn(hidden_size,output_size)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrdereDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],self.params['b1'],conv.params['stride'],conv.params['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2,pool_w=2,stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss()
    
    #순전파
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        return self.last_layer.forward(y,t)
    
    #역전파
    def gradient(self,x,t):
        self.loss(x,t)
        
        dout= 1
        dout = self.last_layer_backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads= {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].dW
        
        return grads